In [1]:
# I've gone ahead and used "=" in place of "<-" because the latest R doesn't
# care, also makes the code more (Pythonic)readable IMHO


# Legend of MAGIC Numbers, File Paths and Arguments
CORRELATION_STRATEGY = "pearson"
PATH_TO_DATA = "~/Git-Projects/spect-team/Data/"
PRIMARY_FILE = "Data_Adults_1.csv"
INFO_START = 2
INFO_END = 14
DISORDER_START = 15
DISORDER_END = 76
SURVEY_START = 77
SURVEY_END = 378
RCBF_RAW_START = 379
RCBF_RAW_END = 636
RCBF_SCALED_START = 637
RCBF_SCALED_END = 754

The above describe some Magic numbers and column indices. They offer very rudimentary insight into what columns map on to what data. While we're at it let us also define a handful of helpful functions. 
**PLEASE FIX THE FILE PATH TO SOMETHING THAT POINTS TO A FILE ON YOUR COMPUTER**

In [4]:
# Computes Correlation Between 2 quantities and returns a correlation vector.
# Param: quant_a : The first data frame to correlate
# Param: quant_b : The second data frame to correlate with 
# Return:  A vector of column correlation
correlate = function(quant_a, quant_b = NA) {
    if (is.na(quant_b)){
        cor_val = cor(quant_a, method = CORRELATION_STRATEGY)
    } else {
        cor_val = cor(quant_a, quant_b, method = CORRELATION_STRATEGY)
    }
    return (cor_val)
}


# This function returns a matrix with 2 rows, one with n smallest values and the other with n largest.
# Param: vec : The vector from which the values need to be extracted
# Param: n : The number of values needed
# Param: req_plot : A switch which will optionally plot the data if required
# Return: A row bound matrix of n smallest and largest values respectively
sort_plot = function(vec, n, req_plot = FALSE) {
    smallest_n = sort(vec)[1:n]
    largest_n = sort(vec, TRUE)[1:n]
    if (req_plot) {
        plot(vec)
    }
    return (rbind(smallest_n, largest_n))
}


# Function prefixes the patient ID column to a data fram for kicks, actually it
# does this so that we can find out who is where.
# Param: start_index : The beginning of the columns to bind the patient ID
# ahead of
# Param: end_index : The end of the columns
# Return: a data frame with the required columns prefixed by patient ID 
prefix_id = function(start_index, end_index) {
    return (cbind(cleansed_data[2], cleansed_data[, start_index : end_index]))
}

### Descriptive Question - 1
What is the size of our training data? What is the length of each feature vector and how many features does each vector have?

### Descriptive Question - 2
What does the presence of Null values indicate? How should they be dealt with?

In [9]:
spect_data = data.frame(read.csv(paste0(PATH_TO_DATA, PRIMARY_FILE), as.is = TRUE))
cleansed_data = na.omit(spect_data) # deal with Null values
cat("The dataset as is has", nrow(spect_data), "rows and", ncol(spect_data), "columns\n")
cat("The downsampled dataset, omitting rows that have NA values has", nrow(cleansed_data), 
    "rows and", ncol(cleansed_data), "columns\n")

The dataset as is has 7674 rows and 754 columns
The downsampled dataset, omitting rows that have NA values has 2796 rows and 754 columns


**Answer 1**: As evidenced by the code above even when we elect to throw out any row that has a NA value we still have a sizeable datset. This doesn't mean however that we will not use the data thrown out. We will simply downsample till we have a complete yet sizeable dataset and draw corellation conclusions etc. from it. Once we have these theories we will try to apply them to less downsampled datasets to see if we can fill in missing data and verify if our theories still hold.

**Answer 2**: As stated in the answer above, NA values were thrown out. We still had a considerably large dataset after doing so. 

### Descriptive Question - 4
What do the features in the vector indicate? What are they for? How are the feature values arrived at?

In [ ]:
cleansed_data_patient_info = cleansed_data[,INFO_START : INFO_END]
cleansed_data_disorder_info = prefix_id(DISORDER_START, DISORDER_END)
cleansed_data_survey_info = prefix_id(SURVEY_START, SURVEY_END)
cleansed_data_RCBF_raw_info = prefix_id(RCBF_RAW_START, RCBF_RAW_END)
cleansed_data_RCBF_scaled_info = prefix_id(RCBF_SCALED_START, RCBF_SCALED_END)

**Answer 4**: By manually examining the data, we realized that certain portions of the dataset contain specific data that can be
examined mutually exclusively from the other portions of the dataset. We made note of the column indices that allow the data to be split into these mutually exclusive portions and went ahead and split the data set according to those indices. This also allows for easier correlation experiments.

- The columns in *patient_info* represent information about the patient like gender, age, location etc
- The columns in *disorder_info* represent a series of boolean values that indicate whether the patient ails
  from a particular disorder
- The columns in *survey_info* represent the patient's responses to the BSC, GSC and LDS survey questions(assuming they are surveys)
- The columns in *RCBF_raw* represent the raw cerebral blood flow values
- The columns in *RCBF_scaled* represent the scaled cerebral blood flow values